In [1]:
import os
import pickle
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import torch.utils.data as data1
from torch.utils import data
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
import torchvision.transforms as transforms
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter

In [2]:
from torch.utils import data
class Dataload_3D_CNN(data.Dataset):
    "Characterizes a dataset for PyTorch"
    def __init__(self, data_path, transform=None):
        "Initialization"
        self.transform = transform
        #self.frames = frames
        self.folders = data_path

    def __len__(self):
        "Denotes the total number of samples"
        return len(os.listdir(self.folders))

    def read_images(self, data_path, use_transform):
        X = []
        for i in os.listdir(data_path):
            #print("file name is ",i)
            image = Image.open(os.path.join(data_path,i))
            
            #print(image.shape)
            if use_transform is not None:
                image = use_transform(image)
                #print(image.size)
            image = torch.from_numpy(np.asarray(image))
            X.append(image)
        #print(X)
        #X = np.array(X)
        X = torch.stack(X, dim=0)

        return X

    def __getitem__(self, index):
        "Generates one sample of data"
        # Select sample
        #print("index passed is ",index)
        #print(self.folders)
        data_path = os.path.join(self.folders,os.listdir(self.folders)[index])
        #data_path2 = self.folders+ str(index)
        #print("Data path is ",data_path2)
        
        # Load data
        X = self.read_images(data_path, self.transform)                     # (input) spatial images
        
        y = 1
        if 'orig' in data_path or 'real' in data_path:
            y = 0
        # print(X.shape)
        return X, torch.from_numpy(np.array(y)).type(torch.LongTensor)

In [3]:
TRANSFORM_IMG_TRAIN = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.RandomResizedCrop(256),
    transforms.ToTensor()
    
    ])

In [4]:
TRANSFORM_IMG_VAL = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor()
])

In [5]:
train_path = '/home/chinmay/datatset/combined_dataset/train/'
train_data = Dataload_3D_CNN(train_path, transform=TRANSFORM_IMG_TRAIN)
# for step, (x, y) in enumerate(data):
#     print(x.shape)
val_path = '/home/chinmay/datatset/combined_dataset/val/'
val_data = Dataload_3D_CNN(val_path, transform=TRANSFORM_IMG_VAL)

In [6]:
epochs = 40
batch_size = 8
learning_rate = 1e-5
log_interval = 10
img_x, img_y = 96,96#128,128#256, 256  # resize video 2d frame size

In [7]:
# Detect devices
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU
print("Is use_cuda", use_cuda)
# Now load the dataset
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
# Load the dataset

train_loader = data1.DataLoader(train_data, **params)
valid_loader = data1.DataLoader(val_data, **params)

Is use_cuda True


In [8]:
def find_median(numpy_array = []): #This is expected to take an array of array. So,
    #print("Input array is {}".format(numpy_array))
    output = []
    confidence_scores = []
    for array in numpy_array:
        counts = np.bincount(array)
        output.append(np.argmax(counts))
        # Let us compute the confidence of the scores
        # since frames are independent, our confidence is purely based on the number
        # of frames our model thinks is belonging to a specific category
        # the confidence of individual frame prediction is not taken into consideration
        # and this portion is debatable....
        frame_set_pred = np.sort(counts)[-1]
        confidence = frame_set_pred/sum(counts)
        confidence_scores.append(confidence)
    return torch.from_numpy(np.asarray(output)).type(torch.LongTensor), torch.from_numpy(np.asarray(confidence_scores)).type(torch.FloatTensor)

In [9]:
# Model paths
writer_train = SummaryWriter('/home/chinmay/training-results/conv3D_combined/train/')
writer_test = SummaryWriter('/home/chinmay/training-results/conv3D_combined/test/')
save_model_path = "/home/chinmay/model_weights/conv3D_combined/"


loss_fn = nn.CrossEntropyLoss()

# 3D CNN parameters
fc_hidden1, fc_hidden2 = 256, 256
dropout = 0.0        # dropout probability


# Select which frame to begin & end in videos
begin_frame, end_frame, skip_frame = 1, 10, 1

def train(log_interval, model, device, train_loader, optimizer, epoch):
    # set model as training mode
    model.train()

    losses = []
    scores = []
    N_count = 0   # counting total trained sample in one epoch
    for batch_idx, (X, y) in enumerate(train_loader):
        # distribute data to device
        #X, y = X.to(device), y.to(device)
        X, y = X.cuda(), y.cuda()
        #print("The label is ",y)
        N_count += X.size(0)
        #print("The size is ",X.size())
        optimizer.zero_grad()
        output = model(X)  # output size = (batch, number of classes)
        #y, _ = find_median(y) #This is necessary as now only single label output for entire frame
        y = y.to(device)
        #print(y)
        #print(y.shape)
        loss = F.cross_entropy(output, y)
        losses.append(loss.item())

        # to compute accuracy
        y_pred = torch.max(output, 1)[1]  # y_pred != output
        step_score = accuracy_score(y.cpu().data.squeeze().numpy(), y_pred.cpu().data.squeeze().numpy())
        scores.append(step_score)         # computed on CPU

        loss.backward()
        optimizer.step()
        
                 
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item(), 100 * step_score))
         
    return np.mean(losses), np.mean(scores)

In [10]:
def validation(model, device, optimizer, test_loader):
    # set model as testing mode
    model.eval()

    all_y = []
    all_y_pred = []
    test_loss = []
    with torch.no_grad():
        for X, y in test_loader:
            # distribute data to device
            #X, y = X.to(device), y.to(device)
            X, y = X.cuda(), y.cuda()
            output = model(X)

            #y, _ = find_median(y) #This is necessary as now only single label output for entire frame
            y = y.to(device)
            loss = F.cross_entropy(output, y)
            test_loss.append(loss.item())                 # sum up batch loss
            y_pred = output.max(1, keepdim=True)[1]  # (y_pred != output) get the index of the max log-probability
            
            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)

    test_loss = np.mean(test_loss)

    # to compute accuracy
#     all_y = torch.stack(all_y, dim=0)
#     all_y_pred = torch.stack(all_y_pred, dim=0)
    all_y = torch.stack(all_y, dim=0)
    all_y_pred = torch.stack(all_y_pred, dim=0)
    test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())

    # show information
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100* test_score))

    # save Pytorch models of best record
    torch.save(model.state_dict(), os.path.join(save_model_path, 'cnn3d_f2f{}.pth'.format(epoch + 1)))  # save spatial_encoder
    torch.save(optimizer.state_dict(), os.path.join(save_model_path, 'cnn3d_f2f_epoch{}.pth'.format(epoch + 1)))      # save optimizer
    print("Epoch {} model saved!".format(epoch + 1))


    return test_loss, test_score

In [11]:
# create model
#cnn3d = CNN3D(t_dim=10, img_x=img_x, img_y=img_y,
#              drop_p=dropout, fc_hidden1=fc_hidden1,  fc_hidden2=fc_hidden2, num_classes=2)

from Res3D_DF import C3D
cnn3d = C3D(img_dim=256, frames=10, dropout=0.1)

cnn3d.cuda()
# Parallelize model to multiple GPUs
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    cnn3d = nn.DataParallel(cnn3d)

optimizer = torch.optim.Adam(cnn3d.parameters(), lr=learning_rate)   # optimize all cnn parameters


# record training process
epoch_train_losses = []
epoch_train_scores = []
epoch_test_losses = []
epoch_test_scores = []


# start training
for epoch in range(epochs):
    # train, test model
    train_losses, train_scores = train(log_interval, cnn3d, device, train_loader, optimizer, epoch)
    epoch_test_loss, epoch_test_score = validation(cnn3d, device, optimizer, valid_loader)

    # save all train test results
    # save results
    writer_train.add_scalar('loss',train_losses,epoch+1)
    writer_train.add_scalar('score',train_scores,epoch+1)
    writer_test.add_scalar('loss',epoch_test_loss,epoch+1)
    writer_test.add_scalar('score',epoch_test_score,epoch+1)
    
    
    torch.cuda.empty_cache()


Train Epoch: 1 [80/2643 (3%)]	Loss: 5.108966, Accu: 0.00%
Train Epoch: 1 [160/2643 (6%)]	Loss: 4.220590, Accu: 75.00%
Train Epoch: 1 [240/2643 (9%)]	Loss: 3.687297, Accu: 62.50%
Train Epoch: 1 [320/2643 (12%)]	Loss: 3.237847, Accu: 50.00%
Train Epoch: 1 [400/2643 (15%)]	Loss: 2.202766, Accu: 62.50%
Train Epoch: 1 [480/2643 (18%)]	Loss: 1.542899, Accu: 62.50%
Train Epoch: 1 [560/2643 (21%)]	Loss: 1.179708, Accu: 62.50%
Train Epoch: 1 [640/2643 (24%)]	Loss: 0.895830, Accu: 50.00%
Train Epoch: 1 [720/2643 (27%)]	Loss: 1.195800, Accu: 62.50%
Train Epoch: 1 [800/2643 (30%)]	Loss: 1.241175, Accu: 37.50%
Train Epoch: 1 [880/2643 (33%)]	Loss: 1.362922, Accu: 50.00%
Train Epoch: 1 [960/2643 (36%)]	Loss: 1.031048, Accu: 25.00%
Train Epoch: 1 [1040/2643 (39%)]	Loss: 0.995470, Accu: 75.00%
Train Epoch: 1 [1120/2643 (42%)]	Loss: 0.968226, Accu: 62.50%
Train Epoch: 1 [1200/2643 (45%)]	Loss: 0.812548, Accu: 50.00%
Train Epoch: 1 [1280/2643 (48%)]	Loss: 0.823173, Accu: 25.00%
Train Epoch: 1 [1360/2643

Train Epoch: 4 [2480/2643 (94%)]	Loss: 0.783482, Accu: 50.00%
Train Epoch: 4 [2560/2643 (97%)]	Loss: 0.748175, Accu: 37.50%
Train Epoch: 4 [2640/2643 (100%)]	Loss: 0.545393, Accu: 87.50%

Test set (1009 samples): Average loss: 0.6716, Accuracy: 64.42%

Epoch 4 model saved!
Train Epoch: 5 [80/2643 (3%)]	Loss: 0.804622, Accu: 50.00%
Train Epoch: 5 [160/2643 (6%)]	Loss: 0.743149, Accu: 50.00%
Train Epoch: 5 [240/2643 (9%)]	Loss: 0.810142, Accu: 50.00%
Train Epoch: 5 [320/2643 (12%)]	Loss: 0.516003, Accu: 100.00%
Train Epoch: 5 [400/2643 (15%)]	Loss: 0.682388, Accu: 62.50%
Train Epoch: 5 [480/2643 (18%)]	Loss: 0.686010, Accu: 50.00%
Train Epoch: 5 [560/2643 (21%)]	Loss: 0.664338, Accu: 50.00%
Train Epoch: 5 [640/2643 (24%)]	Loss: 0.667119, Accu: 75.00%
Train Epoch: 5 [720/2643 (27%)]	Loss: 0.676615, Accu: 75.00%
Train Epoch: 5 [800/2643 (30%)]	Loss: 0.681130, Accu: 62.50%
Train Epoch: 5 [880/2643 (33%)]	Loss: 0.678340, Accu: 62.50%
Train Epoch: 5 [960/2643 (36%)]	Loss: 0.693472, Accu: 50.0

Train Epoch: 8 [2160/2643 (82%)]	Loss: 0.575595, Accu: 62.50%
Train Epoch: 8 [2240/2643 (85%)]	Loss: 0.730542, Accu: 62.50%
Train Epoch: 8 [2320/2643 (88%)]	Loss: 0.858959, Accu: 37.50%
Train Epoch: 8 [2400/2643 (91%)]	Loss: 0.590819, Accu: 87.50%
Train Epoch: 8 [2480/2643 (94%)]	Loss: 0.618217, Accu: 75.00%
Train Epoch: 8 [2560/2643 (97%)]	Loss: 0.538399, Accu: 87.50%
Train Epoch: 8 [2640/2643 (100%)]	Loss: 0.727834, Accu: 50.00%

Test set (1009 samples): Average loss: 0.6385, Accuracy: 68.19%

Epoch 8 model saved!
Train Epoch: 9 [80/2643 (3%)]	Loss: 0.632981, Accu: 62.50%
Train Epoch: 9 [160/2643 (6%)]	Loss: 0.570656, Accu: 87.50%
Train Epoch: 9 [240/2643 (9%)]	Loss: 0.608152, Accu: 62.50%
Train Epoch: 9 [320/2643 (12%)]	Loss: 0.720558, Accu: 50.00%
Train Epoch: 9 [400/2643 (15%)]	Loss: 0.640831, Accu: 62.50%
Train Epoch: 9 [480/2643 (18%)]	Loss: 1.158493, Accu: 37.50%
Train Epoch: 9 [560/2643 (21%)]	Loss: 0.790058, Accu: 37.50%
Train Epoch: 9 [640/2643 (24%)]	Loss: 0.713269, Accu: 6

Train Epoch: 12 [1760/2643 (66%)]	Loss: 0.725342, Accu: 50.00%
Train Epoch: 12 [1840/2643 (69%)]	Loss: 0.620463, Accu: 75.00%
Train Epoch: 12 [1920/2643 (73%)]	Loss: 0.531930, Accu: 62.50%
Train Epoch: 12 [2000/2643 (76%)]	Loss: 0.495047, Accu: 87.50%
Train Epoch: 12 [2080/2643 (79%)]	Loss: 0.785096, Accu: 50.00%
Train Epoch: 12 [2160/2643 (82%)]	Loss: 0.738949, Accu: 37.50%
Train Epoch: 12 [2240/2643 (85%)]	Loss: 0.832228, Accu: 25.00%
Train Epoch: 12 [2320/2643 (88%)]	Loss: 0.538595, Accu: 87.50%
Train Epoch: 12 [2400/2643 (91%)]	Loss: 0.655171, Accu: 37.50%
Train Epoch: 12 [2480/2643 (94%)]	Loss: 0.560575, Accu: 75.00%
Train Epoch: 12 [2560/2643 (97%)]	Loss: 0.857414, Accu: 50.00%
Train Epoch: 12 [2640/2643 (100%)]	Loss: 0.482351, Accu: 87.50%

Test set (1009 samples): Average loss: 0.7195, Accuracy: 56.49%

Epoch 12 model saved!
Train Epoch: 13 [80/2643 (3%)]	Loss: 0.757641, Accu: 50.00%
Train Epoch: 13 [160/2643 (6%)]	Loss: 0.735594, Accu: 37.50%
Train Epoch: 13 [240/2643 (9%)]	Lo

Train Epoch: 16 [1280/2643 (48%)]	Loss: 0.518561, Accu: 62.50%
Train Epoch: 16 [1360/2643 (51%)]	Loss: 1.043964, Accu: 50.00%
Train Epoch: 16 [1440/2643 (54%)]	Loss: 0.698352, Accu: 37.50%
Train Epoch: 16 [1520/2643 (57%)]	Loss: 0.819364, Accu: 50.00%
Train Epoch: 16 [1600/2643 (60%)]	Loss: 0.528098, Accu: 75.00%
Train Epoch: 16 [1680/2643 (63%)]	Loss: 0.572187, Accu: 62.50%
Train Epoch: 16 [1760/2643 (66%)]	Loss: 0.649968, Accu: 62.50%
Train Epoch: 16 [1840/2643 (69%)]	Loss: 0.882709, Accu: 25.00%
Train Epoch: 16 [1920/2643 (73%)]	Loss: 0.354189, Accu: 100.00%
Train Epoch: 16 [2000/2643 (76%)]	Loss: 0.562772, Accu: 75.00%
Train Epoch: 16 [2080/2643 (79%)]	Loss: 0.723833, Accu: 50.00%
Train Epoch: 16 [2160/2643 (82%)]	Loss: 0.527051, Accu: 75.00%
Train Epoch: 16 [2240/2643 (85%)]	Loss: 0.650083, Accu: 75.00%
Train Epoch: 16 [2320/2643 (88%)]	Loss: 0.497254, Accu: 75.00%
Train Epoch: 16 [2400/2643 (91%)]	Loss: 0.575386, Accu: 62.50%
Train Epoch: 16 [2480/2643 (94%)]	Loss: 0.675696, Accu

Process Process-140:
Process Process-137:
Traceback (most recent call last):
Process Process-139:
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
Process Process-138:
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/chinmay/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._a

KeyboardInterrupt: 